# **Trabajo Practico Numero 2**

*<strong>Alumnos:</strong> Guido Pauletti y Maria Agustina Burgos Pesqueira.*

In [31]:
import numpy as np
import pandas as pd

**Usamos pandas para leer el archivo csv y echarle un vistazo a la forma de los datos**

In [32]:
wines = pd.read_csv('wine.csv')
wines.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


**Notamos algo importante:**<br>
    Las escalas de los diferentes atributos difieren,<br>
    algunas estan en los cientos, y otras varian entre 0 y 1.

In [33]:
wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Alcohol               178 non-null    float64
 1   Malic_Acid            178 non-null    float64
 2   Ash                   178 non-null    float64
 3   Ash_Alcanity          178 non-null    float64
 4   Magnesium             178 non-null    int64  
 5   Total_Phenols         178 non-null    float64
 6   Flavanoids            178 non-null    float64
 7   Nonflavanoid_Phenols  178 non-null    float64
 8   Proanthocyanins       178 non-null    float64
 9   Color_Intensity       178 non-null    float64
 10  Hue                   178 non-null    float64
 11  OD280                 178 non-null    float64
 12  Proline               178 non-null    int64  
 13  Customer_Segment      178 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


<h3>Separo las variables independientes de la dependiente con X e Y</h3>

In [34]:
# guardamos los datos en matrices de numpy
Y = np.array(wines['Customer_Segment'])
X = np.array(wines.drop('Customer_Segment', axis=1))

**Corroboramos que las matrices sean de la forma correcta**

In [35]:
print(f'X: {X.shape[0]} filas; {X.shape[1]} columnas\nY: {Y.shape[0]} filas; 1 columna')

X: 178 filas; 13 columnas
Y: 178 filas; 1 columna


<h3>Creamos una funcion para normalizar los datos</h3>

**Primero** definimos una funcion para obtener la desviacion standard muestral de cada variable<br>
 independiente, se pasan como parametros un vector represetando los valores de las variables<br>
  para cada instancia, y tambien la media, necesaria para calcular el radicando.

Se hara uso de una funcion auxiliar $desviacion\_standard\_muestral$ para trabajar con
cada columna de la matriz. <br>
* **Parametro**: vector, lista, o array de numpy.
* **Devuelve**: la desviacion standard muestral $(s)$ calculada con los elementos del parametro.

Siendo el vector de la forma: $[X_1,X_2,...,X_n]$<br>
Para calcular el resultado se usan las siguientes formulas:<br><br>
$media = (\sum_{i=1}^{n} X_i)/n$<br><br>
$s = \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} (X_i - media)^{2}}$

In [36]:
def desviacion_standard(vector):

    # calculamos la media
    media = sum(vector) / len(vector)

    # inicializamos sumatoria, que llevara la suma de los cuadrados de las diferencias entre las instancias y la media
    sumatoria = 0
    for i in range(len(vector)):
        # recorremos las instancias para sumar las diferencias
        sumatoria += (vector[i] - media)**2

    # aplicamos la formula para obtener la desviacion standard
    s = np.sqrt(sumatoria / (len(vector) - 1))
    return (s, media)

**Luego** definimos la funcion principal, <br>
toma una matriz como parametro y la devuelve con sus valores normalizados

In [37]:
def normalizar(matriz):
    # obtenemos la forma de la matriz
    filas, columnas = matriz.shape
    
    # recorremos las columnas (los atributos de cada vino)
    for i in range(columnas):
        columna_actual = matriz[:,i]
        
        # calculamos la desviacion standard del atributo actual
        s, media = desviacion_standard(columna_actual)

        # recorremos los valores del atributo y los vamos normalizando
        for f in range(filas):
            matriz[f,i] = (matriz[f,i] - media) / s

    # devolvemos la matriz normalizada
    return matriz

In [38]:
X = normalizar(X)

Ahora tengo la matriz con los valores normalizados, esto es importante<br>
como paso previo a bucar covarianzas entre los atributos de los vinos. <br><br>
Como habiamos visto en el dataframe, hay columnas con escalas mayores a<br>
otras, lo que iba a ser una complicación al momento de decidir si una<br>
relacion entre dos atributos es fuerte o debil.

<h3>Obtenemos la matriz de covarianza</h3>

Para entender mejor de que se trata, supongamos que solo tenemos<br>
tres atributos de los vinos: $x$, $y$ y $z$. La matriz de covarianza tendrá la forma:<br><br>
$
\begin{pmatrix}
var_{x} & con_{xy} & cov_{xz}\\
cov_{xy} & var_{y} & cov_{yz}\\
cov_{xz} & con_{zy} & var_{z}
\end{pmatrix}
$

Por lo que la matriz de covarianza es una matriz cuadrada y simetrica.<br>
Teniendo en la diagonal las varianzas de cada atributo.

In [39]:
X = np.cov(X, rowvar=False)

# vemos que es una matriz cuadrada
print(f'La dimension de la matriz de covarianza es {X.shape}')

La dimension de la matriz de covarianza es (13, 13)


<h3>Aplicamos el metodo de la potenica</h3>
Y encontramos el maximo autovector con su autovalor

In [42]:
def n_autovectores(A):

    # genero un vector random para aplicar el algoritmo de la potencia
    v = np.random.rand(A.shape[0])

    # itero 100 veces (arbitrario)
    for i in range(100):
        Av = A @ v
        v = Av / np.linalg.norm(Av, 2)

    autovector = v
    autovalor = np.linalg.norm(A @ v, 2)

    return(autovector, autovalor)


In [43]:
avector, avalor = n_autovectores(X)
print(f"autovalor maximo: {avector}\nautovalor: {avalor}")

autovalor maximo: [ 0.1443294  -0.24518758 -0.00205106 -0.23932041  0.14199204  0.39466085
  0.4229343  -0.2985331   0.31342949 -0.0886167   0.29671456  0.37616741
  0.28675223]
autovalor: 4.705850252990424


Lo siguiente seria entender el metodo PCA (Principal Component Analysis), <br>
para que quede bien en claro porque me sirve sacar los autovectores de la matriz.<br><br>
En <a href="https://es.wikipedia.org/wiki/An%C3%A1lisis_de_componentes_principales">esta pagina</a> encontré informacion util. Sobre todo estos parrafos:<br>

<blockquote>
El ACP se emplea sobre todo en análisis exploratorio de datos y para construir modelos predictivos. El ACP comporta el cálculo de la descomposición en autovalores de la matriz de covarianza, normalmente tras centrar los datos en la media de cada atributo.
</blockquote><br>
<blockquote>
En el análisis de datos, el primer componente principal de un conjunto de 
p variables, que se supone están distribuidas normalmente de forma conjunta, es la variable derivada formada como combinación lineal de las variables originales que explica la mayor parte de la varianza. El segundo componente principal explica la mayor varianza de lo que queda una vez eliminado el efecto del primer componente, y podemos proceder a través de p iteraciones hasta que se explique toda la varianza.
</blockquote><br>
<blockquote>
Para cualquier objetivo, se puede demostrar que los componentes principales son vectores propios de la matriz de covarianza de los datos. Por lo tanto, los componentes principales se calculan a menudo por eigendecomposition de la matriz de covarianza de datos o descomposición en valores singulares de la matriz de datos. 
</blockquote>